In [2]:
import sys
import os
import importlib

sys.path.insert(0, os.path.abspath(".."))

if "filter_utils" in sys.modules:
    importlib.reload(sys.modules["filter_utils"])

from scr.filter_utils import *
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt

In [3]:
image_selector = widgets.Dropdown(
    options=["Generated Sample"] + get_available_images(),
    value="Generated Sample",
    description="Image:",
    style={"description_width": "80px"},
    layout=widgets.Layout(width="300px"),
)

upload_button = widgets.FileUpload(
    accept="image/*",
    description="Upload Image",
    multiple=False,
    layout=widgets.Layout(width="200px"),
)

image_output = widgets.Output()
selected_image = None


def load_selected_image():
    global selected_image
    if image_selector.value == "Generated Sample":
        selected_image = load_sample_image()
    else:
        image_path = f"data/img/{image_selector.value}"
        selected_image = load_image_from_path(image_path)

    with image_output:
        image_output.clear_output(wait=True)
        fig, ax = plt.subplots(1, 1, figsize=(6, 6))
        ax.imshow(selected_image, cmap="gray")
        ax.set_title("Selected Image")
        ax.axis("off")
        plt.tight_layout()
        plt.show()


def on_image_change(change):
    load_selected_image()


def on_upload(change):
    if len(upload_button.value) > 0:
        global selected_image
        import io

        uploaded_file = upload_button.value[0]
        content = io.BytesIO(uploaded_file["content"])
        img = Image.open(content).convert("L")
        img.thumbnail((512, 512), Image.Resampling.LANCZOS)
        selected_image = np.array(img) / 255.0

        with image_output:
            image_output.clear_output(wait=True)
            fig, ax = plt.subplots(1, 1, figsize=(6, 6))
            ax.imshow(selected_image, cmap="gray")
            ax.set_title("Uploaded Image")
            ax.axis("off")
            plt.tight_layout()
            plt.show()


image_selector.unobserve_all()
upload_button.unobserve_all()

image_selector.observe(on_image_change, names="value")
upload_button.observe(on_upload, names="value")

load_selected_image()

display(
    widgets.VBox(
        [
            widgets.HBox(
                [image_selector, upload_button],
                layout=widgets.Layout(gap="20px", justify_content="center"),
            ),
            widgets.HTML("<br>"),
            image_output,
        ],
        layout=widgets.Layout(padding="20px", align_items="center"),
    )
)

In [ ]:
create_filter_pipeline_widget(selected_image)